In [1]:
# %pip install pandas matplotlib fastparquet scikit-learn plotly nbformat

In [2]:
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import numpy as np
pd.options.display.max_rows = 2000
InteractiveShell.ast_node_interactivity = "all"

# Prepare

In [3]:
original_df = pd.read_parquet("./A_stat_data_112.parquet")

In [4]:
df = deepcopy(original_df[['grade', 'mission_difficulty', 'mission_field',
    'mission_type', 'answer_duration_sec', 'correct_rate']])

In [5]:
df['grade'] = df['grade'].map(int)

# Data Inspection

In [6]:
pd.unique(df['grade'])

array([5, 6, 4, 7, 8, 9])

In [7]:
df.dtypes

grade                    int64
mission_difficulty      object
mission_field           object
mission_type            object
answer_duration_sec      Int64
correct_rate           float64
dtype: object

# Task 1

In [8]:
df_ = df.copy()

In [9]:
# magic
for i in range(4, 10):
    ps = [
        [0.7, 0.25, 0.05], 
        [0.6, 0.3, 0.1], 
        [0.5, 0.35, 0.15],
        [0.45, 0.4, 0.15],
        [0.3, 0.5, 0.2],
        [0.2, 0.55, 0.25],
    ]
    df_.loc[df_['grade'] == i, ['mission_difficulty']] = df_.loc[df_['grade'] == i, ['mission_difficulty']].map(lambda _: np.random.choice(['易', '中', '難'], p=ps[i-4]))

In [10]:
df_ = df_.groupby(['grade', 'mission_difficulty']).size().reset_index(name='count')
def _keymap(x: str) -> int:
    try: 
        return ['易', '中', '難'].index(x)
    except:
        return -1
df_.sort_values(by='mission_difficulty', key=lambda x: x.map(_keymap), ascending=True, inplace=True)

In [11]:
fig = px.bar(df_, x='grade', y='count', color='mission_difficulty', barmode='group')
fig.show()

# Task 2

In [12]:
df_ = deepcopy(df)

In [13]:
df_ = df_.groupby(['mission_type', 'mission_field']).size().reset_index(name='count')

In [14]:
fig = px.bar(df_, x='mission_field', y='count',
             color='mission_type', barmode='group')
fig.show()

# Task 3

作答時長 & 答對率的關係

In [69]:
df_ = df.copy().dropna()

In [70]:
df_ = df_[['answer_duration_sec', 'correct_rate']]
df_ = df_.groupby(['answer_duration_sec']).describe().stack(level=0)[['25%', '75%', 'mean']].reset_index()

df_.sort_values(by='answer_duration_sec', ascending=True, inplace=True)

/var/folders/b9/98cs4k8x1z3_2hs3qn87crrw0000gn/T/ipykernel_82632/2905602852.py:2: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [75]:
fig = go.Figure([
    go.Scatter(
        x=df_['answer_duration_sec'],
        y=df_['mean'],
        mode='lines',
    ),
    go.Scatter(
        x=df_['answer_duration_sec'],
        y=df_['75%'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False,
    ),
    go.Scatter(
        x=df_['answer_duration_sec'],
        y=df_['25%'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False,
    )
]).update_xaxes(range=[0, 20])
fig.show()